# Задание 5 - 20 баллов

- Загрузить набор данных Lenta.ru с помощью пакета Corus
- Обучить LDA модель, постараться подобрать адекватные параметры (num_topics, passes, alpha, iterations…) - **4 балла**
- Визуализировать результаты работы LDA с помощью pyLDAvis - **2 балла**
- Посчитать внутренние метрики обученных моделей LDA (с разными параметрами) и сравнить, соответствует ли метрика визуальному качеству работы моделей - **2 балла**
- Обучить модель BigARTM, использовать не менее двух регуляризаторов, оценить качество с помощью метрик - **5 баллов**
- Реализовать визуализацию топиков BigARTM через pyLDAvis - **4 балла**

- Обеспечена воспроизводимость решения: зафиксированы random_state, ноутбук воспроизводится от начала до конца без ошибок - **2 балла**

- Соблюден code style на уровне pep8 и [On writing clean Jupyter notebooks](https://ploomber.io/blog/clean-nbs/)  - **1 балл**

Примечание: подбирать параметры теметической модели можно также, как и для любой другой модели - на кроссвалидации, ориентируясь на метрики качества

In [1]:
from corus import load_lenta
import os
import gensim

from typing import List

from itertools import product

import demoji
import re
import spacy

from tqdm import tqdm

from gensim import corpora

import pandas as pd
from pprint import pprint

import numpy as np

from sklearn.feature_extraction.text import CountVectorizer
from gensim.models.coherencemodel import CoherenceModel

import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
import artm


NLP = spacy.load("ru_core_news_sm")
CHECK_POS = {'PUNCT', 'ADP', 'AUX', 'CCONJ', 'SCONJ'}
PATTERN_EMAIL = r'([A-Za-z0-9]+[.-_])*[A-Za-z0-9]+@[A-Za-z0-9-]+(\.[A-Z|a-z]{2,})+'

SEED = 566
# data dir
DATA_DIR = "../data"
# path to the data
DATA_PATH = os.path.join(DATA_DIR, "lenta-ru-news.csv.gz")
# amount of texts to work with
N_TEXTS = 500

/home/toharhymes/work/itmo/nlp_2023fall/venv/lib/python3.10/site-packages/pymorphy3/analyzer.py:114: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  import pkg_resources
/home/toharhymes/work/itmo/nlp_2023fall/venv/lib/python3.10/site-packages/pkg_resources/__init__.py:2871: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declare_namespace(pkg)


# Загрузка текста

Загрузим данные в директорию `../data/`.

Также для сохранения визуализаций должна присутствовать директория `../images/`

In [6]:
# ! wget https://github.com/yutkin/Lenta.Ru-News-Dataset/releases/download/v1.0/lenta-ru-news.csv.gz
# ! mv lenta-ru-news.csv.gz ../data/

## Читаем датасет

In [2]:
records = load_lenta(DATA_PATH)
next(records)

LentaRecord(
    url='https://lenta.ru/news/2018/12/14/cancer/',
    title='Названы регионы России с\xa0самой высокой смертностью от\xa0рака',
    text='Вице-премьер по социальным вопросам Татьяна Голикова рассказала, в каких регионах России зафиксирована наиболее высокая смертность от рака, сообщает РИА Новости. По словам Голиковой, чаще всего онкологические заболевания становились причиной смерти в Псковской, Тверской, Тульской и Орловской областях, а также в Севастополе. Вице-премьер напомнила, что главные факторы смертности в России — рак и болезни системы кровообращения. В начале года стало известно, что смертность от онкологических заболеваний среди россиян снизилась впервые за три года. По данным Росстата, в 2017 году от рака умерли 289 тысяч человек. Это на 3,5 процента меньше, чем годом ранее.',
    topic='Россия',
    tags='Общество',
    date=None
)

In [3]:
dataset_text_topics = [next(records) for i in range(N_TEXTS)]
dataset = [tt.text for tt in dataset_text_topics]
target = [tt.topic for tt in dataset_text_topics]
dataset[0], target[0]

('Австрийские правоохранительные органы не представили доказательств нарушения российскими биатлонистами антидопинговых правил. Об этом сообщил посол России в Вене Дмитрий Любинский по итогам встречи уполномоченного адвоката дипмиссии с представителями прокуратуры страны, передает ТАСС. «Действует презумпция невиновности. Каких-либо ограничений свободы передвижения для команды нет», — добавили в посольстве. Международный союз биатлонистов (IBU) также не будет применять санкции к российским биатлонистам. Все они продолжат выступление на Кубке мира. Полиция нагрянула в отель сборной России в Хохфильцене вечером 12 декабря. Как написал биатлонист Александр Логинов, их считают виновными в махинациях с переливанием крови. Биатлонисту Антону Шипулину, также попавшему в список, полиция нанесла отдельный визит: сейчас он тренируется отдельно в австрийском Обертиллахе. Обвинения спортсмен назвал бредом, а также указал на «охоту на ведьм» в мировом биатлоне. В Австрии прием допинга — уголовное п

## Предобработка

In [4]:
def normalize_text(text: str) -> str:
    checked_text = demoji.replace(text)
    checked_text = re.sub(PATTERN_EMAIL, '<EMAIL>', checked_text)
    return checked_text


def normalize_data(text_data: List[str]) -> List[str]:
    return list(map(normalize_text, text_data))


def tokenize_clean_stem(text: str) -> List[str]:
    text_NLPed = NLP(text)
    return [token.lemma_ for token in text_NLPed
            if token.pos_ not in CHECK_POS and not token.is_stop]


def tokenize_clean_stem_data(text_data: List[str]) -> List[List[str]]:
    return list(map(tokenize_clean_stem, text_data))


def pipeline_preprocessing(text_data: List[str]) -> List[List[str]]:
    data = normalize_data(text_data)
    return tokenize_clean_stem_data(data)


In [5]:
%%time
preprocessed_dataset = pipeline_preprocessing(dataset)

CPU times: user 23.5 s, sys: 22.4 ms, total: 23.5 s
Wall time: 23.5 s


In [6]:
preprocessed_dataset[0][:10], len(preprocessed_dataset[0])

(['австрийский',
  'правоохранительный',
  'орган',
  'представить',
  'доказательство',
  'нарушение',
  'российский',
  'биатлонист',
  'антидопинговый',
  'правило'],
 100)

## Получение N-grams

`Automatically detect common phrases – aka multi-word expressions, word n-gram collocations – from a stream of sentences.`

In [7]:
def get_ngrams(texts_out, bigram_mod, trigram_mod):
    texts_out = [bigram_mod[doc] for doc in texts_out]
    texts_out = [trigram_mod[bigram_mod[doc]] for doc in texts_out]
    return texts_out

In [8]:
bigram = gensim.models.Phrases(preprocessed_dataset, min_count=5, threshold=100)  # higher threshold fewer phrases.
bigram[preprocessed_dataset][0][:5], bigram[preprocessed_dataset][5][5:10]

(['австрийский',
  'правоохранительный_орган',
  'представить',
  'доказательство',
  'нарушение'],
 ['ценность', 'распространять', 'интернет', 'социальный_сеть', 'инициатива'])

Некоторы фразочки объединились, и в целом звучат логично: 'правоохранительный_орган', 'социальный_сеть'

In [9]:
trigram = gensim.models.Phrases(bigram[preprocessed_dataset], threshold=100)
# make Phraser == FrozenPhrases
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)
tokenized_dataset = get_ngrams(preprocessed_dataset, bigram_mod, trigram_mod)
tokenized_dataset[0][:10]

['австрийский',
 'правоохранительный_орган',
 'представить',
 'доказательство',
 'нарушение',
 'российский',
 'биатлонист',
 'антидопинговый_правило',
 'сообщить',
 'посол']

In [10]:
flag = 10
for item in tokenized_dataset:
    for token in item:
        if token.count('_') > 2:
            print(token)
            flag -= 1
    if flag < 0:
        break

московский_патриархат_упц_мп
заговор_цель_ведение_деятельность
московский_патриархат_упц_мп
московский_патриархат_упц_мп
московский_патриархат_упц_мп
московский_патриархат_упц_мп
московский_патриархат_упц_мп
московский_патриархат_упц_мп
московский_патриархат_упц_мп
заговор_цель_ведение_деятельность
релиз_поступить_редакция_ленты.ру


## Создание словаря

В конце у нас: bag-of-words format = list of (token_id, token_count)

In [11]:
id2word = corpora.Dictionary(tokenized_dataset)
dict(id2word)[1], dict(id2word)[5]

('ibu', 'антидопинговый_правило')

In [12]:
corpus = [id2word.doc2bow(text) for text in tokenized_dataset]
corpus[0][:10]

[(0, 1),
 (1, 1),
 (2, 2),
 (3, 1),
 (4, 1),
 (5, 1),
 (6, 1),
 (7, 1),
 (8, 3),
 (9, 1)]

# LDA модель

In [40]:
%%time
nums_topics = [3, 5, 7]
alphas = ['symmetric', 'asymmetric']  # A-priori belief on document-topic distribution
passes = [25, 100]  # Number of passes through the corpus during training
iterations = [25,
              100]  # Maximum number of iterations through the corpus when inferring the topic distribution of a corpus

params_sets = list(product(nums_topics, alphas, passes, iterations))
param_names = ['num_topics', 'alpha', 'passes', 'iterations']

coherence_scores = dict()
perplexity_scores = dict()
models = dict()

for num_topics, alpha, passes, iterations in tqdm(params_sets):
    lda_model = gensim.models.ldamodel.LdaModel(
        corpus=corpus,
        id2word=id2word,
        num_topics=num_topics,
        random_state=SEED,
        update_every=1,  # Number of documents to be iterated through for each update
        chunksize=10,  # == batch size
        passes=passes,  # Number of passes through the corpus during training
        alpha=alpha,  # A-priori belief on document-topic distribution
        iterations=iterations,
        # Maximum number of iterations through the corpus when inferring the topic distribution of a corpus,
    )

    coherence_model_lda = CoherenceModel(model=lda_model,
                                         texts=tokenized_dataset,
                                         dictionary=id2word,
                                         coherence='c_v')
    coherence_score = coherence_model_lda.get_coherence()

    perplexity_score = lda_model.log_perplexity(corpus)

    coherence_scores[(num_topics, alpha, passes, iterations)] = coherence_score
    perplexity_scores[(num_topics, alpha, passes, iterations)] = perplexity_score
    models[(num_topics, alpha, passes, iterations)] = lda_model

100%|██████████| 24/24 [05:18<00:00, 13.29s/it]

CPU times: user 6min 1s, sys: 0 ns, total: 6min 1s
Wall time: 5min 18s


Let's look at scores

In [41]:
df_coherence = pd.DataFrame(list(coherence_scores.items()), columns=['Params', 'Coherence'])
df_perplexity = pd.DataFrame(list(perplexity_scores.items()), columns=['Params', 'Log_Perplexity'])
df = pd.merge(df_coherence, df_perplexity, on='Params', how='outer')
df['Perplexity'] = df['Log_Perplexity'].apply(lambda x: 10 ** x)
df_sorted = df.sort_values(by=['Coherence'], ascending=[False], na_position='last')
display(df_sorted)

,Params,Coherence,Log_Perplexity,Perplexity
16,"(7, symmetric, 25, 25)",0.371087,-9.474258,3.355381e-10
18,"(7, symmetric, 100, 25)",0.368366,-9.154316,7.009452e-10
12,"(5, asymmetric, 25, 25)",0.363092,-9.264435,5.439577e-10
2,"(3, symmetric, 100, 25)",0.357287,-8.864147,1.367267e-09
10,"(5, symmetric, 100, 25)",0.353174,-9.031985,9.289974e-10
20,"(7, asymmetric, 25, 25)",0.348206,-9.489662,3.238454e-10
0,"(3, symmetric, 25, 25)",0.347581,-9.029243,9.348818e-10
6,"(3, asymmetric, 100, 25)",0.342348,-8.869061,1.351881e-09
4,"(3, asymmetric, 25, 25)",0.338784,-9.025110,9.438222e-10
8,"(5, symmetric, 25, 25)",0.338635,-9.257704,5.524537e-10


Перплексия везде маленькая, и примерно одинаковая (как и когерентность). Возьмем по когерентности лучшую модель.

In [42]:
best_params = (7, 'symmetric', 25, 25)
best_lda_model = models[best_params]
pprint(best_lda_model.print_topics())

[(0,
  '0.014*"проект" + 0.010*"учёный" + 0.008*"деньга" + 0.008*"система" + '
  '0.008*"цель" + 0.007*"эксперт" + 0.007*"политика" + 0.007*"специалист" + '
  '0.005*"журнал" + 0.005*"несколько"'),
 (1,
  '0.046*"процент" + 0.016*"год" + 0.014*"2019" + 0.010*"30" + 0.009*"газ" + '
  '0.007*"миллион" + 0.006*"рост" + 0.006*"уровень" + 0.006*"банк" + '
  '0.005*"процент_опросить"'),
 (2,
  '0.028*"украина" + 0.011*"развитие" + 0.009*"зарплата" + 0.009*"принять" + '
  '0.007*"депутат" + 0.007*"мероприятие" + 0.007*"конституция" + 0.007*"союз" '
  '+ 0.007*"современный" + 0.007*"отказаться"'),
 (3,
  '0.023*"-" + 0.018*"год" + 0.015*"россия" + 0.009*"заявить" + '
  '0.009*"российский" + 0.009*"страна" + 0.009*"слово" + 0.007*"компания" + '
  '0.007*"время" + 0.006*"сша"'),
 (4,
  '0.011*"обнаружить" + 0.011*"турция" + 0.008*"запретить" + '
  '0.007*"пользователь" + 0.007*"смерть" + 0.006*"робот" + 0.006*"google" + '
  '0.006*"исследование" + 0.006*"женщина" + 0.005*"защита"'),
 (5,
  '0.01

Визуализируем:

In [43]:
pyLDAvis.enable_notebook()
vis = gensimvis.prepare(best_lda_model, corpus, dictionary=best_lda_model.id2word)
pyLDAvis.save_html(vis, '../images/lda_best.html')
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.312300 -0.171159       1        1  37.732943
5      0.154795  0.280280       2        1  24.088601
2     -0.095501 -0.038817       3        1   8.380284
0     -0.114491 -0.007304       4        1   8.372683
4     -0.095155 -0.028901       5        1   7.972490
1     -0.077709 -0.015163       6        1   7.071391
6     -0.084239 -0.018937       7        1   6.381608, topic_info=             Term        Freq       Total Category  logprob  loglift
836       процент  195.000000  195.000000  Default  30.0000  30.0000
18            год  669.000000  669.000000  Default  29.0000  29.0000
0               -  783.000000  783.000000  Default  28.0000  28.0000
417       украина  139.000000  139.000000  Default  27.0000  27.0000
62         россия  337.000000  337.000000  Default  26.0000  26.0000
...           ...         ...         ...      ...      ...      ...
656          море   14.143578   14.913816   Topic7  -5.5982   2.6987
7040  командующий   14.962075   15.779380   Topic7  -5.5420   2.6986
497           ход   34.649659   49.407438   Topic7  -4.7022   2.3969
674          сила   19.039590   33.652919   Topic7  -5.3010   2.1822
61     российский   16.319387  221.314198   Topic7  -5.4551   0.1445

[283 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
0         1  0.663339         -
0         2  0.336772         -
4191      6  0.962692       1,5
3392      5  0.957448         2
2985      4  0.973636      2015
...     ...       ...       ...
2653      4  0.974227       шоу
140       4  0.972140   эксперт
684       1  0.896943  являться
684       3  0.032034  являться
684       7  0.064067  являться

[286 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 6, 3, 1, 5, 2, 7])

Если визуализация в данный момент не отображается в гитхабе -- есть html: [images/lda_best.html](images/lda_best.html)

### Сравнение с худшей

Давайте нарисуем то же самое для худшей из списка модели, и посмотрим как у нее с визуализацией.

In [45]:
worst_params = (7, 'asymmetric', 100, 25)
worst_lda_model = models[worst_params]
pprint(worst_lda_model.print_topics())

[(0,
  '0.016*"-" + 0.016*"год" + 0.008*"проект" + 0.008*"2018" + 0.006*"миллион" + '
  '0.006*"сайт" + 0.005*"четыре" + 0.005*"объект" + 0.005*"ход" + '
  '0.005*"работать"'),
 (1,
  '0.041*"процент" + 0.015*"год" + 0.013*"2019" + 0.009*"30" + 0.009*"вызвать" '
  '+ 0.009*"газ" + 0.007*"представить" + 0.007*"рост" + 0.006*"банк" + '
  '0.006*"уровень"'),
 (2,
  '0.024*"украина" + 0.011*"государство" + 0.009*"развитие" + 0.007*"принять" '
  '+ 0.007*"требование" + 0.007*"отказаться" + 0.006*"зарплата" + '
  '0.006*"мероприятие" + 0.006*"депутат" + 0.006*"современный"'),
 (3,
  '0.019*"-" + 0.016*"россия" + 0.015*"год" + 0.010*"российский" + '
  '0.010*"заявить" + 0.009*"слово" + 0.009*"страна" + 0.008*"компания" + '
  '0.007*"президент" + 0.007*"сша"'),
 (4,
  '0.010*"обнаружить" + 0.009*"турция" + 0.008*"учёный" + 0.006*"робот" + '
  '0.006*"смерть" + 0.006*"женщина" + 0.005*"исследование" + 0.005*"операция" '
  '+ 0.005*"google" + 0.005*"защита"'),
 (5,
  '0.015*"-" + 0.011*"год" + 0

In [46]:
pyLDAvis.enable_notebook()
vis = gensimvis.prepare(worst_lda_model, corpus, dictionary=worst_lda_model.id2word)
pyLDAvis.save_html(vis, '../images/lda_worst.html')
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3     -0.307226  0.171831       1        1  34.021414
5     -0.142005 -0.276542       2        1  22.873097
0      0.040066 -0.039597       3        1  14.353237
2      0.108776  0.064192       4        1   9.074139
4      0.114943  0.029880       5        1   7.509878
1      0.095355  0.031183       6        1   6.789529
6      0.090090  0.019054       7        1   5.378706, topic_info=            Term        Freq       Total Category  logprob  loglift
836      процент  168.000000  168.000000  Default  30.0000  30.0000
62        россия  332.000000  332.000000  Default  29.0000  29.0000
417      украина  132.000000  132.000000  Default  28.0000  28.0000
18           год  636.000000  636.000000  Default  27.0000  27.0000
0              -  733.000000  733.000000  Default  26.0000  26.0000
...          ...         ...         ...      ...      ...      ...
667        полёт   11.583418   12.316955   Topic7  -5.6269   2.8613
1642     предмет   11.869896   12.629622   Topic7  -5.6025   2.8607
308          ряд   14.912118   21.213362   Topic7  -5.3743   2.5703
61    российский   16.314730  227.202223   Topic7  -5.2844   0.2890
674         сила   12.517878   37.436577   Topic7  -5.5493   1.8272

[290 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
0         1  0.527881         -
0         2  0.279627         -
0         3  0.192329         -
4191      6  0.955136       1,5
3392      5  0.925305         2
...     ...       ...       ...
139       3  0.981152    четыре
336       1  0.991172     число
3983      3  0.989851    чёрный
140       3  0.988527   эксперт
684       1  0.995591  являться

[297 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 6, 1, 3, 5, 2, 7])

Если визуализация в данный момент не отображается в гитхабе -- есть html: [images/lda_worst.html](images/lda_worst.html)

В обоих случаях выделяется два кластера: при этом в худшем по когерентности, еще один кластер выделяется, все остальные же достаточно похожи по частотам (судя по PCA) -- поэтому скорее всего естественным было бы  использование количества кластеров равного трем, а не 7ми. Давайте отрисуем картинку для набора с наилучшей когерентность из данных с количеством кластеров = 3 -- одновременно это датасет с бОльшей (если можно сравнивать такие маленькие числа) перплексией.

In [48]:
three_params = (3, 'symmetric', 100, 25)
three_lda_model = models[three_params]
pprint(three_lda_model.print_topics())

[(0,
  '0.014*"год" + 0.014*"-" + 0.009*"россия" + 0.006*"российский" + '
  '0.005*"процент" + 0.005*"страна" + 0.004*"заявить" + 0.004*"слово" + '
  '0.004*"компания" + 0.004*"украина"'),
 (1,
  '0.010*"-" + 0.006*"год" + 0.005*"сообщать" + 0.005*"мужчина" + '
  '0.004*"человек" + 0.004*"ребёнок" + 0.004*"время" + 0.003*"место" + '
  '0.003*"женщина" + 0.003*"имя"'),
 (2,
  '0.005*"учёный" + 0.003*"спортсмен" + 0.003*"являться" + 0.003*"чёрный" + '
  '0.003*"бренд" + 0.002*"сборная" + 0.002*"коллекция" + 0.002*"цвет" + '
  '0.002*"\xa0" + 0.002*"исследователь"')]


In [49]:
pyLDAvis.enable_notebook()
vis = gensimvis.prepare(three_lda_model, corpus, dictionary=three_lda_model.id2word)
pyLDAvis.save_html(vis, '../images/lda_best_three.html')
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.193069  0.025903       1        1  60.246883
1      0.069005 -0.149199       2        1  29.782569
2      0.124064  0.123296       3        1   9.970547, topic_info=            Term        Freq       Total Category  logprob  loglift
62        россия  314.000000  314.000000  Default  30.0000  30.0000
61    российский  219.000000  219.000000  Default  29.0000  29.0000
836      процент  193.000000  193.000000  Default  28.0000  28.0000
545      мужчина   84.000000   84.000000  Default  27.0000  27.0000
684     являться   79.000000   79.000000  Default  26.0000  26.0000
...          ...         ...         ...      ...      ...      ...
528        земля    9.214166   15.394977   Topic3  -6.4730   1.7922
152                12.573679   58.182391   Topic3  -6.1621   0.7736
1290       пресс   11.227507   66.006272   Topic3  -6.2753   0.5342
527       защита    9.402230   26.055525   Topic3  -6.4527   1.2863
559   обнаружить   10.009697   47.088036   Topic3  -6.3901   0.7571

[181 rows x 6 columns], token_table=       Topic      Freq      Term
term                            
0          1  0.729007         -
0          2  0.270816         -
773        1  0.977666      2019
773        2  0.016294      2019
10667      3  0.873729    armani
...      ...       ...       ...
3983       2  0.059411    чёрный
3983       3  0.950575    чёрный
684        1  0.753218  являться
684        3  0.238519  являться
5102       3  0.933017    яндекс

[202 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 3])

Если визуализация в данный момент не отображается в гитхабе -- есть html: [images/lda_best_three.html](images/lda_best_three.html)

Далее, можно подробно изучать частоты внутри каждой из групп...:)

# BigARTM
Обучить модель BigARTM, использовать не менее двух регуляризаторов, оценить качество с помощью метрик.

## Векторизация
Объединим токенизированные тексты и векторизуем.

In [20]:
cv = CountVectorizer(max_features=1000, max_df=0.8, min_df=2, ngram_range=(1, 2))
n_wd = np.array(cv.fit_transform([' '.join(text) for text in tokenized_dataset]).todense()).T
token_list = [i for i in cv.vocabulary_.keys()]
bv = artm.BatchVectorizer(data_format='bow_n_wd', n_wd=n_wd, vocabulary=token_list)

/home/toharhymes/work/itmo/nlp_2023fall/venv/lib/python3.10/site-packages/artm/batches_utils.py:227: DeprecationWarning: Please use `spmatrix` from the `scipy.sparse` namespace, the `scipy.sparse.base` namespace is deprecated.
  from scipy.sparse.base import spmatrix


## Train model

In [36]:
def fit_model_simple(seed=SEED):
    model = artm.ARTM(num_topics=10, dictionary=bv.dictionary, cache_theta=True, seed=seed)
    model.scores.add(artm.PerplexityScore(name='perplexity_score', dictionary=bv.dictionary))
    model.scores.add(artm.SparsityPhiScore(name='sparsity_phi_score'))
    model.scores.add(artm.SparsityThetaScore(name='sparsity_theta_score'))
    model.scores.add(artm.TopTokensScore(name='top_tokens_score', num_tokens=100))
    # Регуляризаторы
    # Sparse Word Regularizer (Sparsity Phi Regularizer): Encourages sparsity in the word-topic distribution. It can lead to topics that are more interpretable because each topic will contain fewer and more distinct words.
    model.regularizers.add(
        artm.SmoothSparsePhiRegularizer(
            name='SparsePhi',
            tau=-0.5
        ),
    )   
    # Sparse Topic Regularizer (Sparsity Theta Regularizer): This encourages sparsity in the document-topic distribution. It can make the topics in the documents more specific by having fewer topics per document.
    model.regularizers.add(
        artm.SmoothSparseThetaRegularizer(
            name='SparseTheta',
            tau=-0.5
        ),
    )

    # Decorrelator Regularizer: Decreases the correlation between topics. It encourages the topics to be more different from each other, which can lead to a better distributed representation over the corpus.
    model.regularizers.add(
        artm.DecorrelatorPhiRegularizer(
            name='DecorrelatorPhi',
            tau=0.7
        ), )

    model.fit_offline(bv, num_collection_passes=30)
    return model


In [37]:
%%time
model = fit_model_simple()
for sc in ["perplexity_score", 'sparsity_phi_score', 'sparsity_theta_score']:
    print(f"{sc}: {np.round(model.score_tracker[sc].last_value, 3)}")

perplexity_score: 433.668
sparsity_phi_score: 0.73
sparsity_theta_score: 0.559
CPU times: user 520 ms, sys: 3.95 ms, total: 524 ms
Wall time: 465 ms


Перплексия достаточно большая -- ее можно улучшить, начав подбирать параметры, но опустим это. Скоры по Phi и Theta средние, однако показывают хорошую разреженность слов в темах и документах.

Давайте визуализируем.

In [38]:
def prepare_vis_data():
    phi = model.get_phi()
    theta = model.get_theta().to_numpy().T
    theta = theta / theta.sum(axis=1, keepdims=1)
    data = {'topic_term_dists': phi.to_numpy().T,
            'doc_topic_dists': theta,
            'doc_lengths': n_wd.sum(axis=0).tolist(),
            'vocab': phi.T.columns,
            'term_frequency': n_wd.sum(axis=1).tolist()}
    return data
     

model_data = prepare_vis_data()
model_vis = pyLDAvis.prepare(**model_data)
pyLDAvis.save_html(model_vis, '../images/artm.html')
pyLDAvis.display(model_vis)

/home/toharhymes/work/itmo/nlp_2023fall/venv/lib/python3.10/site-packages/pandas/core/internals/blocks.py:366: RuntimeWarning: divide by zero encountered in log
  result = func(self.values, **kwargs)
/home/toharhymes/work/itmo/nlp_2023fall/venv/lib/python3.10/site-packages/pandas/core/internals/blocks.py:366: RuntimeWarning: divide by zero encountered in log
  result = func(self.values, **kwargs)
/home/toharhymes/work/itmo/nlp_2023fall/venv/lib/python3.10/site-packages/pandas/core/internals/blocks.py:366: RuntimeWarning: divide by zero encountered in log
  result = func(self.values, **kwargs)


BigART разделил тексты абсолютно по-другому (не в три кластера) -- и это действительно так, кластеров то было больше.

Если визуализация в данный момент не отображается в гитхабе -- есть html: [images/artm.html](images/artm.html)